In [147]:
import _adjustpaths
from pymatgen.io.espresso.outputs import PWxml


pristine_file = '/home/fes33/Documents/GIK - R&D/Personal - Papers and Reports/__Shared/SharedTemp/Naseeb/LGPS/aiida.xml'
def struct_from_xml(file = None, step = 0):
    pristine_calc = PWxml(file)
    structure = pristine_calc.ionic_steps[step]['structure']    
    return structure

from pymatgen.core import Structure
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer

def ineq_sites_from_struct(struct, elem = 'Ge', frac = True, verbose = True):
    """
    Return inequivalent sites from struct corresponding to elem.
    """

    sga = SpacegroupAnalyzer(structure)
    symm_struct = sga.get_symmetrized_structure()
    
    equiv_sites = symm_struct.equivalent_sites
    symm_inequivalent_sites = [group[0] for group in equiv_sites if group[0].species_string == elem]

    if frac: 
        for i, site in enumerate(symm_inequivalent_sites):
            if verbose: 
                print(f"Symmetry-inequivalent {elem} atom {i+1}: {site.frac_coords}")
            

    else:
        for i, site in enumerate(symm_inequivalent_sites):
            if verbose: 
                print(f"Symmetry-inequivalent {elem} atom {i+1}: {site.coords}")
    return symm_inequivalent_sites


from pymatgen.core import Lattice, Structure
from pymatgen.symmetry.groups import SpaceGroup
def struct_from_sites_and_sg(lattice = None, species = None, frac_coords = None, spacegroup = "Ccca:2", coords_are_cartesian = True):
    """
    Build structure from sites and spacegroup
    """
    a,b,c,alpha,beta,gamma = cell_param 

    # Step 1: Define lattice
    # Replace with actual lattice vectors or parameters
    # lattice = Lattice.from_parameters(a,b, c, alpha, beta, gamma)  # put correct values here

    # Step 3: Generate structure from space group
    structure = Structure.from_spacegroup(
        sg=spacegroup,          # Space group 68, origin choice 2
        lattice=lattice,
        species=species,
        coords=frac_coords,
        coords_are_cartesian=coords_are_cartesian
    )

    
    return structure


from pymatgen.core import Structure

def remove_elems_from_struct(structure: Structure, elements_to_remove: list[str]) -> Structure:
    """
    Return a new Structure with specified elements removed.

    Args:
        structure (Structure): Input pymatgen Structure.
        elements_to_remove (list[str]): List of element symbols to remove.

    Returns:
        Structure: New Structure without the specified elements.
    """
    # Find site indices to remove
    indices_to_remove = [
        i for i, site in enumerate(structure)
        if str(site.specie) in elements_to_remove
    ]
    # Copy and remove
    new_structure = structure.copy()
    new_structure.remove_sites(indices_to_remove)
    
    return new_structure


In [156]:
new_struct = remove_elems_from_struct(structure, ['Li', 'S', 'P'])


In [157]:
new_struct


Structure Summary
Lattice
    abc : 18.856766793093843 18.856766793093776 13.591587804337502
 angles : 90.0 90.0 141.9028466491571
 volume : 2981.8620683978033
      A : np.float64(17.824211948374373) np.float64(6.1542767494284) np.float64(0.0)
      B : np.float64(-17.824211948374405) np.float64(6.154276749428088) np.float64(0.0)
      C : np.float64(0.0) np.float64(0.0) np.float64(13.591587804337502)
    pbc : True True True
PeriodicSite: Ge (-2.952, 3.348, 8.321) [0.1892, 0.3548, 0.6122]
PeriodicSite: Ge (2.952, 2.806, 8.321) [0.3108, 0.1452, 0.6122]
PeriodicSite: Ge (2.952, 3.348, 12.07) [0.3548, 0.1892, 0.8878]
PeriodicSite: Ge (-2.952, 2.806, 12.07) [0.1452, 0.3108, 0.8878]
PeriodicSite: Ge (2.952, 8.961, 5.27) [0.8108, 0.6452, 0.3878]
PeriodicSite: Ge (-2.952, 9.502, 5.27) [0.6892, 0.8548, 0.3878]
PeriodicSite: Ge (-2.952, 8.961, 1.526) [0.6452, 0.8108, 0.1122]
PeriodicSite: Ge (2.952, 9.502, 1.526) [0.8548, 0.6892, 0.1122]
PeriodicSite: Ge (13.19, 6.765, 6.131) [0.9195, 0.1797,

In [158]:
new_ineq = ineq_sites_from_struct(new_struct)

Symmetry-inequivalent Ge atom 1: [0.18917795 0.35480371 0.61224295]
Symmetry-inequivalent Ge atom 2: [0.91946079 0.17972657 0.45106749]


In [159]:

lattice = new_struct.lattice.as_dict()['matrix']

species =[site.species_string for site in new_ineq]
frac_coords = [site.frac_coords for site in new_ineq] 
spacegroup = "Ccca:2"

spacegroup = 68
stucture_recon = struct_from_sites_and_sg(lattice =  lattice, 
                                          species = species,
                                          frac_coords = frac_coords, 
                                          spacegroup = spacegroup,
                                          coords_are_cartesian = False)

ValueError: Supplied lattice with parameters (18.856766793093843, 18.856766793093776, 13.591587804337502, 90.0, 90.0, 141.9028466491571) is incompatible with supplied spacegroup Ccce!

In [ ]:
structure = struct_from_xml(pristine_file, step = 3)
ineq_sites = ineq_sites_from_struct(structure, elem = 'Li', frac = True, verbose = False)
structure

In [152]:

# a, b, c = 17.824211948374373, 6.1542767494284, 13.591587804337502
# alpha, beta, gamma = 90,90,90
# a = 35.5
# b = 12.6
# c = 13.5
# alpha = 90
# beta = 90
# gamma = 90
#cell_param = (a,b,c,alpha,beta,gamma)



lattice = new_struct.lattice.as_dict()['matrix']

species =[site.species_string for site in ineq_sites]
frac_coords = [site.frac_coords for site in ineq_sites] 
spacegroup = "Ccca:2"

spacegroup = 68
stucture_recon = struct_from_sites_and_sg(lattice =  lattice, 
                                          species = species,
                                          frac_coords = frac_coords, 
                                          spacegroup = spacegroup,
                                          coords_are_cartesian = False)

ValueError: Supplied lattice with parameters (18.856766793093843, 18.856766793093776, 13.591587804337502, 90.0, 90.0, 141.9028466491571) is incompatible with supplied spacegroup Ccce!

In [117]:
stucture_recon

Structure Summary
Lattice
    abc : 35.5 12.6 13.5
 angles : 90.0 90.0 90.0
 volume : 6038.55
      A : np.float64(35.5) np.float64(0.0) np.float64(2.173748068486552e-15)
      B : np.float64(2.026234538592562e-15) np.float64(12.6) np.float64(7.715274834628325e-16)
      C : np.float64(0.0) np.float64(0.0) np.float64(13.5)
    pbc : True True True
PeriodicSite: Li (15.5, 11.7, 3.083) [0.4365, 0.9288, 0.2284]
PeriodicSite: Li (2.254, 0.8975, 3.083) [0.0635, 0.07123, 0.2284]
PeriodicSite: Li (2.254, 11.7, 9.833) [0.0635, 0.9288, 0.7284]
PeriodicSite: Li (20.0, 7.197, 3.083) [0.5635, 0.5712, 0.2284]
PeriodicSite: Li (20.0, 5.403, 9.833) [0.5635, 0.4288, 0.7284]
PeriodicSite: Li (2.254, 5.403, 3.667) [0.0635, 0.4288, 0.2716]
PeriodicSite: Li (33.25, 5.403, 3.083) [0.9365, 0.4288, 0.2284]
PeriodicSite: Li (15.5, 7.197, 3.667) [0.4365, 0.5712, 0.2716]
PeriodicSite: Li (33.25, 7.197, 9.833) [0.9365, 0.5712, 0.7284]
PeriodicSite: Li (2.254, 7.197, 10.42) [0.0635, 0.5712, 0.7716]
PeriodicSite: 

In [6]:
structure = pristine_calc.ionic_steps[0]['structure']
elem = 'Ge'
ge_sites = [site for site in structure if site.species_string == elem]
len(ge_sites)

16

In [47]:
structure.lattice.parameters

(18.856766793093843,
 18.856766793093776,
 13.591587804337502,
 90.0,
 90.0,
 141.9028466491571)

In [46]:
dir(structure.lattice)

['REDIRECT',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__firstlineno__',
 '__format__',
 '__ge__',
 '__get_pydantic_core_schema__',
 '__get_pydantic_json_schema__',
 '__get_validators__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__modify_schema__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__static_attributes__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_calculate_lll',
 '_diags',
 '_generic_json_schema',
 '_inv_matrix',
 '_lll_inverse',
 '_lll_matrix_mappings',
 '_matrix',
 '_pbc',
 '_validate_monty',
 'a',
 'abc',
 'alpha',
 'angles',
 'as_dict',
 'b',
 'beta',
 'c',
 'copy',
 'cubic',
 'd_hkl',
 'dot',
 'find_all_mappings',
 'find_mapping',
 'from_dict',
 'from_parameters',
 'gamma',
 'get_all_distances',
 'get_brillouin_zone',
 'get_cartesian_coords',
 'get_distance_and_image',
 'ge

In [44]:
dir(structure)

['CellType',
 'DISTANCE_TOLERANCE',
 'REDIRECT',
 '__abstractmethods__',
 '__annotations__',
 '__class__',
 '__class_getitem__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__firstlineno__',
 '__format__',
 '__ge__',
 '__get_pydantic_core_schema__',
 '__get_pydantic_json_schema__',
 '__get_validators__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__modify_schema__',
 '__module__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__rmul__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__slots__',
 '__static_attributes__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_calculate',
 '_charge',
 '_generic_json_schema',
 '_get_neighbor_list_py',
 '_lattice',
 '_prep_calculator',
 '_properties',
 '_relax',
 '_sites',
 '_validate_monty'

In [49]:
structure.get_space_group_info()

('Ccce', 68)

In [53]:
symm_data = structure.get_symmetry_dataset()

In [54]:
structure

Structure Summary
Lattice
    abc : 18.856766793093843 18.856766793093776 13.591587804337502
 angles : 90.0 90.0 141.9028466491571
 volume : 2981.8620683978033
      A : np.float64(17.824211948374373) np.float64(6.1542767494284) np.float64(0.0)
      B : np.float64(-17.824211948374405) np.float64(6.154276749428088) np.float64(0.0)
      C : np.float64(0.0) np.float64(0.0) np.float64(13.591587804337502)
    pbc : True True True
PeriodicSite: Ge (-2.952, 3.348, 8.321) [0.1892, 0.3548, 0.6122]
PeriodicSite: Ge (2.952, 2.806, 8.321) [0.3108, 0.1452, 0.6122]
PeriodicSite: Ge (2.952, 3.348, 12.07) [0.3548, 0.1892, 0.8878]
PeriodicSite: Ge (-2.952, 2.806, 12.07) [0.1452, 0.3108, 0.8878]
PeriodicSite: Ge (2.952, 8.961, 5.27) [0.8108, 0.6452, 0.3878]
PeriodicSite: Ge (-2.952, 9.502, 5.27) [0.6892, 0.8548, 0.3878]
PeriodicSite: Ge (-2.952, 8.961, 1.526) [0.6452, 0.8108, 0.1122]
PeriodicSite: Ge (2.952, 9.502, 1.526) [0.8548, 0.6892, 0.1122]
PeriodicSite: S (-1.306, 4.747, 8.697) [0.3491, 0.4223,